This notebook is to contain notes on different models, leading up to a decision on which model to use in production.

This notebook incorporates the dataloader defined in https://github.com/axelth/us8kdata , and will also incorporate the feature extraction pipeline prototyped in notebooks/.... and implemented in hasp/{features.py, pipeline.py} etc.

imports for individual models are performed in their respective section to make it easier to see which code to transfer to the library after model selection

In [3]:
%load_ext autoreload
%autoreload 2

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.metrics import classification_report, confusion_matrix

from us8kdata.loader import UrbanSound8K
from hasp.feature_pipeline import make_feature_pipeline
from hasp.feature_pipeline import make_oversampled_feature_pipeline
from hasp.util import combine_classes

In [73]:
data = UrbanSound8K('../hasp/data')
train_fold = [samples for samples in data.fold_audio_generator(fold=[1, 2, 3, 6, 7, 8, 9, 10])]
val_fold = [samples for samples in data.fold_audio_generator(fold=[4])]
test_fold = [samples for samples in data.fold_audio_generator(fold=[5])]
train_y = data.filter_metadata(fold=[1, 2, 3, 6, 7, 8, 9, 10]).classID
val_y = data.filter_metadata(fold=[4]).classID
test_y = data.filter_metadata(fold=[5]).classID

In [33]:
data.metadata[['classID', 'class']]

,classID,class
0,3,dog_bark
1,2,children_playing
2,2,children_playing
3,2,children_playing
4,2,children_playing
...,...,...
8727,1,car_horn
8728,1,car_horn
8729,1,car_horn
8730,1,car_horn


In [6]:
feature_pipe = make_feature_pipeline()
feature_pipe

Pipeline(steps=[('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1034ed670>,
                                     kw_args={'fmax': None, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 512,
                                              'n_mfcc': 20, 'sr': 16000})),
                ('scaler', StandardScaler())])

In [7]:
train_pre = feature_pipe.fit_transform(train_fold)
val_pre = feature_pipe.transform(val_fold)
#test_pre = feature_pipe.transform(test_fold)

# Single sample classification
Model architectures for treating each audio file as a single example and classifying it in one go.

## Logistic Regression (baseline)

### LASSO

## KNN

## SVC

In [10]:
from sklearn.svm import SVC
model_svc = SVC(C=4)
model_svc.fit(train_pre, train_y)

SVC(C=4)

In [18]:
#val_pre.shape
svc_val_prediction = model_svc.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], svc_val_prediction)
))
print(classification_report(
     val_y, 
    svc_val_prediction))
pd.DataFrame(confusion_matrix(val_y, svc_val_prediction), 
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.57      0.42      0.49        59
           7       0.92      0.96      0.94       765
           8       0.91      0.82      0.86       166

    accuracy                           0.90       990
   macro avg       0.80      0.73      0.76       990
weighted avg       0.90      0.90      0.90       990

              precision    recall  f1-score   support

           0       0.52      0.43      0.47       100
           1       0.57      0.42      0.49        59
           2       0.47      0.42      0.44       100
           3       0.41      0.75      0.53       100
           4       0.51      0.63      0.56       100
           5       0.31      0.22      0.26       107
           6       0.50      0.61      0.55        38
           7       0.19      0.04      0.07       120
           8       0.91      0.82      0.86       166
           9       0.28      0.47      0.35       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,43,0,7,12,24,0,0,3,0,11
1,0,25,1,6,8,3,1,6,1,8
2,10,4,42,17,2,0,5,2,0,18
3,0,0,11,75,0,0,4,0,6,4
4,6,3,1,3,63,9,1,6,0,8
5,4,1,1,20,0,24,12,0,5,40
6,0,0,2,4,2,0,23,4,0,3
7,13,9,7,0,22,38,0,5,0,26
8,0,0,5,25,0,0,0,0,136,0
9,7,2,13,23,3,3,0,1,1,47


## RandomForest

## XGBoost

In [14]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()
model_xgb.fit(train_pre, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [19]:
xgb_val_prediction = model_xgb.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_val_prediction)
))
print(classification_report(
     val_y, 
    xgb_val_prediction))
pd.DataFrame(confusion_matrix(val_y, xgb_val_prediction),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.67      0.37      0.48        59
           7       0.92      0.94      0.93       765
           8       0.80      0.83      0.81       166

    accuracy                           0.89       990
   macro avg       0.80      0.71      0.74       990
weighted avg       0.88      0.89      0.88       990

              precision    recall  f1-score   support

           0       0.20      0.08      0.11       100
           1       0.67      0.37      0.48        59
           2       0.42      0.41      0.41       100
           3       0.46      0.75      0.57       100
           4       0.47      0.55      0.50       100
           5       0.35      0.26      0.30       107
           6       0.47      0.66      0.55        38
           7       0.28      0.10      0.15       120
           8       0.80      0.83      0.81       166
           9       0.26      0.49      0.34       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,8,3,22,7,15,9,0,2,8,26
1,2,22,2,7,5,1,0,8,1,11
2,9,1,41,14,10,0,5,1,2,17
3,1,0,5,75,6,0,6,0,4,3
4,1,5,2,4,55,4,0,15,1,13
5,4,1,2,6,0,28,17,0,14,35
6,0,0,4,5,0,0,25,4,0,0
7,9,0,2,0,27,33,0,12,0,37
8,0,0,4,24,0,0,0,0,137,1
9,6,1,14,21,0,4,0,1,4,49


### XGBoost with oversampled features


In [78]:
oversampled_pipe = make_oversampled_feature_pipeline()
oversampled_pipe

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                                     kw_args={'fmax': None, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 512,
                                              'n_mfcc': 20, 'sr': 16000})),
                ('scaler', StandardScaler()), ('estimator', None)])

In [65]:
oversampled_pipe.get_params()

{'memory': None,
 'steps': [('pad',
   FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                       kw_args={'max_len': 64000})),
  ('over_sampler', AugmentingRandomOversampler(sampling_strategy='minority')),
  ('mean_mfcc',
   FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                       kw_args={'fmax': None, 'fmin': 0.0, 'hop_length': 128,
                                'n_fft': 512, 'n_mfcc': 20, 'sr': 16000})),
  ('scaler', StandardScaler()),
  ('estimator', None)],
 'verbose': False,
 'pad': FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                     kw_args={'max_len': 64000}),
 'over_sampler': AugmentingRandomOversampler(sampling_strategy='minority'),
 'mean_mfcc': FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                     kw_args={'fmax': None, 'fmin': 0.0, 'hop_length': 128,
                              'n_fft': 512, 'n_mfcc': 20, 'sr': 16000}),
 'sc

In [82]:
oversampled_pipe.set_params(estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),                          
                           mean_mfcc__kw_args={'fmin':300, 'fmax':4000})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                                     kw_args={'fmax': 4000, 'fmin': 300})),
                ('scaler', StandardScaler()),
                ('estimator',
                 XGB...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [83]:
%%time
oversampled_pipe.fit(train_fold, train_y);

CPU times: user 4min 55s, sys: 35.4 s, total: 5min 30s
Wall time: 1min 4s


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                                     kw_args={'fmax': 4000, 'fmin': 300})),
                ('scaler', StandardScaler()),
                ('estimator',
                 XGB...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [84]:
xgb_ovsamp_val_pred = oversampled_pipe.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_ovsamp_val_pred)
))
print(classification_report(
     val_y, 
    xgb_ovsamp_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_ovsamp_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.39      0.34      0.36        59
           7       0.91      0.92      0.91       765
           8       0.82      0.80      0.81       166

    accuracy                           0.87       990
   macro avg       0.71      0.69      0.69       990
weighted avg       0.86      0.87      0.86       990

              precision    recall  f1-score   support

           0       0.39      0.17      0.24       100
           1       0.39      0.34      0.36        59
           2       0.39      0.43      0.41       100
           3       0.50      0.79      0.61       100
           4       0.50      0.57      0.54       100
           5       0.33      0.24      0.28       107
           6       0.61      0.61      0.61        38
           7       0.20      0.07      0.11       120
           8       0.82      0.80      0.81       166
           9       0.26      0.50      0.34       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,17,13,30,1,5,10,0,2,4,18
1,2,20,1,6,5,3,0,8,0,14
2,7,0,43,14,12,0,4,2,0,18
3,2,1,2,79,3,0,4,0,6,3
4,1,4,2,3,57,5,1,18,0,9
5,6,9,3,3,0,26,6,0,16,38
6,0,0,6,4,0,0,23,4,0,1
7,6,1,4,0,29,31,0,9,0,40
8,0,0,5,25,1,0,0,0,132,3
9,3,3,13,23,1,3,0,1,3,50


#### oversampled gridsearch

In [85]:
mean_mfcc_grid = []
for n_mfcc in [13, 16, 20]:#range(13, 26):
    for n_fft in range(256, 1025, 256):
        for fmin in [0.0, 300.0, 500.0]: #50.0, 100.0, 200.0, 300.0, 400.0, 500.0]:
            for fmax in [None, 3000.0, 4000.0, 5000.0]: #, 4000.0, 3000.0]:
                mean_mfcc_grid.append({'sr':16000, 'n_mfcc': n_mfcc, 'n_fft': n_fft,
                                       'hop_length': 128, 'fmin': fmin, 'fmax': fmax})
len(mean_mfcc_grid)

144

In [86]:
xgb_grid = [
    XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=8, objective='multi:softmax'),
    XGBClassifier(n_estimators=400, learning_rate=0.025, max_depth=8, objective='multi:softmax')
]

In [87]:
from sklearn.model_selection import GridSearchCV

In [91]:
%%time
gs = GridSearchCV(oversampled_pipe, 
                  dict(mean_mfcc__kw_args=mean_mfcc_grid,
                      estimator=xgb_grid), 
                  cv=2, verbose=2, n_jobs=1)
gs_res = gs.fit(train_fold, train_y)

Fitting 2 folds for each of 288 candidates, totalling 576 fits


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 13, 'n_fft': 256, 'hop_length': 128, 'fmin': 0.0, 'fmax': None}; total time=  51.1s


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 13, 'n_fft': 256, 'hop_length': 128, 'fmin': 0.0, 'fmax': None}; total time=  50.5s


...

[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 20, 'n_fft': 1024, 'hop_length': 128, 'fmin': 500.0, 'fmax': 5000.0}; total time= 1.2min


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=960
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=800
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=960
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 20, 'n_fft': 1024, 'hop_length': 128, 'fmin': 500.0, 'fmax': 5000.0}; total time= 1.3min


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


CPU times: user 2d 7h 28min 45s, sys: 13h 3min 21s, total: 2d 20h 32min 7s
Wall time: 9h 9min 21s


In [93]:
gs_res.best_params_

{'estimator': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.05, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=400, n_jobs=None,
               num_parallel_tree=None, objective='multi:softmax', predictor=None,
               random_state=None, reg_alpha=None, ...),
 'mean_mfcc__kw_args': {'sr': 16000,
  'n_mfcc': 16,
  'n_fft': 256,
  'hop_length': 128,
  'fmin': 0.0,
  'fmax': 3000.0}}

In [94]:
xgb_gs_val_pred = gs.best_estimator_.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_gs_val_pred)
))
print(classification_report(
     val_y, 
    xgb_gs_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_gs_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


              precision    recall  f1-score   support

           1       0.39      0.53      0.45        59
           7       0.92      0.91      0.91       765
           8       0.84      0.79      0.81       166

    accuracy                           0.86       990
   macro avg       0.72      0.74      0.72       990
weighted avg       0.87      0.86      0.87       990

              precision    recall  f1-score   support

           0       0.47      0.20      0.28       100
           1       0.39      0.53      0.45        59
           2       0.40      0.49      0.44       100
           3       0.50      0.77      0.60       100
           4       0.63      0.54      0.58       100
           5       0.37      0.24      0.29       107
           6       0.75      0.63      0.69        38
           7       0.24      0.07      0.11       120
           8       0.84      0.79      0.81       166
           9       0.24      0.49      0.32       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,20,10,32,0,13,4,0,0,0,21
1,2,31,1,5,8,1,0,3,1,7
2,4,0,49,17,3,3,5,1,0,18
3,0,0,7,77,2,0,3,0,6,5
4,2,7,1,5,54,1,0,20,0,10
5,6,1,9,2,0,26,0,0,12,51
6,0,0,5,4,0,0,24,3,0,2
7,4,24,4,0,5,30,0,9,0,44
8,0,0,5,30,0,0,0,0,131,0
9,5,6,11,15,1,6,0,1,6,49


In [95]:
post_search_pipeline = make_oversampled_feature_pipeline()
post_search_pipeline.set_params(**gs.best_params_)
post_search_pipeline.fit(train_fold, train_y)

/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x1634f1310>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1635e2d30>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 256,
                                              'n_mfcc': 16, 'sr'...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.05, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=0, num_parallel_tree=1,
                               objective='multi:softmax', predictor='auto',
                               random_state=0, reg_alpha=0, ...))])

In [96]:
xgb_ps_val_pred = post_search_pipeline.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_ps_val_pred)
))
print(classification_report(
     val_y, 
    xgb_ps_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_ps_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


              precision    recall  f1-score   support

           1       0.39      0.54      0.45        59
           7       0.92      0.91      0.91       765
           8       0.86      0.78      0.82       166

    accuracy                           0.86       990
   macro avg       0.72      0.74      0.73       990
weighted avg       0.88      0.86      0.87       990

              precision    recall  f1-score   support

           0       0.48      0.21      0.29       100
           1       0.39      0.54      0.45        59
           2       0.39      0.47      0.43       100
           3       0.49      0.77      0.60       100
           4       0.62      0.55      0.58       100
           5       0.39      0.24      0.30       107
           6       0.73      0.63      0.68        38
           7       0.27      0.09      0.14       120
           8       0.86      0.78      0.82       166
           9       0.25      0.51      0.34       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,21,11,31,0,8,3,0,2,0,24
1,2,32,1,4,7,0,0,4,1,8
2,5,0,47,19,6,2,6,1,0,14
3,0,1,6,77,3,0,3,0,5,5
4,2,4,2,6,55,2,0,19,0,10
5,6,11,8,4,0,26,0,0,10,42
6,0,0,5,4,0,0,24,3,0,2
7,3,17,4,0,10,28,0,11,0,47
8,0,0,6,29,0,0,0,0,130,1
9,5,6,11,15,0,5,0,1,6,51


## VGGish

# Sequence of samples classification
Model architectures for treating each file as a sequence of sample frames, making the classification either based on a subset or on the frame as a whole.

## HMM

## RNN